# General approach
- Make all terms lowercase
- Identify German nouns with a list and convert these to upper case

## Challenges


## Step 1: Read Excel and convert everything in the 'object' column to  lower case


In [1]:
import pandas as pd

df = pd.read_excel('ELSST_R3_DE.xlsx')
print(len(df.index))

count = range(1,len(df.index))

for i in count:
    old = df['object'][i]
    new = old.lower()
    df = df.replace(old,new)

df.to_excel('forELSSTGroup\GER_ELSST_lowercase.xlsx', sheet_name='Sheet 1')


241


## Step 2 - Compile a list of nouns and verbs
I used the csv-list offered here: https://pypi.org/project/german-nouns/. It was compiled from WiktionaryDE. The list had a number of issues and I have done a lot of trimming to it to improve it based on the results I received. Among the issues were very short nouns, names of cities, etc. The outcome of this is a text file with each noun (including different forms) on an individual line. 
I ended up compiling a list of verbs too ensure that verbs weren't capizalized, but the results are sketchy. 
 

## Step 3 - Set the beginning of the definitions and the scope notes to uppercase


In [2]:
import openpyxl

wb = openpyxl.load_workbook("forELSSTGroup\GER_ELSST_lowercase.xlsx")
ws = wb["Sheet 1"]

for r in range(1,ws.max_row+1):
    label = ws.cell(r,3).value
    terms = ws.cell(r,4).value
    if label == "definition_de" or label == "scopeNote_de":
        ws.cell(r,4).value = ws.cell(r,4).value.replace(terms[0], terms[0].capitalize(),1)

wb.save("forELSSTGroup\GER_ELSST_definitions.xlsx")



## Step 4 - Capitalize all words in the list of nouns and NOT in the list of verbs


In [3]:
import openpyxl
import re

with open('wortliste/neue_nomenliste.txt', 'r', encoding="utf8") as f:
    nomen = [line.strip() for line in f]

with open('wortliste/neue_verben.txt', 'r', encoding="utf8") as f:
    verben = [line.strip() for line in f]    # to do replace ö,ü,ä,

wb = openpyxl.load_workbook("forELSSTGroup\GER_ELSST_definitions.xlsx")
ws = wb["Sheet 1"]

for wort in nomen:
    for r in range(1,ws.max_row+1):            
        terms = ws.cell(r,4).value
        label = ws.cell(r,3).value
        for item in terms.split(' '):             
            if not re.search(r"\W",item): # identifies every string that doesn't have a comma, parentheses or other non-word character in it
                if item != item.capitalize() and item not in verben:
                    if item == wort.lower():
                            to_replace = r"\b" + item + r"\b"
                            ws.cell(r,4).value = re.sub(to_replace, item.capitalize(),ws.cell(r,4).value)
 
            else:
                if re.search(r"^[(']",item): # checks if the string begins with parantheses or single quotation marks
                    if wort.lower() == item[1:-1] and item[1:-1] not in verben:
                        ws.cell(r,4).value = re.sub(re.escape(item[1:-1]),re.escape(wort),ws.cell(r,4).value)
                            #print(f"New value for (': {ws.cell(r,4).value}")
                    elif wort.lower() == item[1:-2]:
                        ws.cell(r,4).value = re.sub(re.escape(item[1:-2]),re.escape(wort),ws.cell(r,4).value)
             
                else:
                    if re.search(r"[\.,]",item): # checks if the string has a comma or full stop
                        if wort.lower() == item[:-1] and item[:-1] not in verben:
                            to_replace = re.search(r"\b"+item[:-1],item)
                            if to_replace:
                                ws.cell(r,4).value = re.sub(r"\b"+to_replace.group()+r"\b",wort,ws.cell(r,4).value)

wb.save('forELSSTGroup\GER_ELSST_targetfile1.xlsx')

## Step 5 - Capitalize all words which end in typical noun suffixes (heit, keit, etc.)

In [5]:
import openpyxl
import re

wb = openpyxl.load_workbook("forELSSTGroup\GER_ELSST_targetfile1.xlsx")
ws = wb["Sheet 1"]

heit = ["ing","heit","heiten", "keit","keiten", "ung", "ungen" ,"nis", "nisse","nissen", "schaft", "schaften", "tum","tümer"]

for endung in heit:
    endungen = endung + r"\b"
    for r in range(1,ws.max_row+1):
    #for r in range(1,50):
        terms = ws.cell(r,4).value
        for item in terms.split(' '):
            if not re.search('\W',item):
                if item != item.capitalize():
                    #print(item)
                    match = re.search(endungen, item)
                    if match:
                        #print(item)
                        ws.cell(r,4).value = re.sub(item, item.capitalize(),ws.cell(r,4).value)
                        #print(ws.cell(r,4).value)
wb.save('forELSSTGroup\GER_ELSST_targetfile2.xlsx') 


## Step 6: Capitalize compounds ending in a noun

In [7]:
import openpyxl
import re

#nomen = ['Größe','Grösse','Verein', 'Vereine','Vereinigung','Vereinigungen','Dienstleistung','Dienstleistungen','Gruppe','Gruppen','Aengste', 'Angststoerungen', 'Angststörungen', 'Angstzustaende', 'Angstzustände', 'Anhaenger', 'Anhänger', 'Anrufe', 'Ausbildung', 'Ausbildung', 'Badminton', 'Befähigung', 'Begriff', 'Beschaeftigung', 'Beschäftigung', 'Bestimmung', 'Bildung', 'Bildung', 'Boxen', 'Bundespraesidenten', 'Bundespräsidenten', 'Chancen', 'Darlehen', 'Depression', 'Depressionen', 'Desinformation', 'Diagnose', 'Diagnosen', 'Diagnostik', 'Dienstleistungen', 'Downer', 'Energie', 'Energieeffizienz', 'Energieeinsparung', 'Ergebnisse', 'Ernährung', 'Erwartung', 'Erwartungen', 'Fachgebiete', 'Falschmeldung', 'Falschmeldungen', 'Fans', 'Fernstudium', 'Fernunterricht', 'Flugverkehr', 'Freimaurerei', 'Gebuehren', 'Gebühren', 'Gesundheitswesen', 'Gruppen', 'Haustiere', 'Heroin', 'Herrscher', 'Interessen', 'Judo', 'Kampfkunst', 'Kampfsport', 'Kandidaten', 'Kandidatin', 'Kindern', 'Klassenarbeiten', 'Klausuren', 'Koexistenz', 'Kolonialismus', 'Krankengeschichte', 'Krankenhauspatienten', 'Krankheit', 'Krieg', 'Kriege', 'Kriegfuehrung', 'Kriegführung', 'Kriegsfuehrung', 'Kriegsführung', 'Labortests', 'Lastautos', 'Lastkraftwagen', 'Lastwagen', 'Lehrberuf', 'Lehrtaetigkeit', 'Lehrtätigkeit', 'Linguistik', 'Lkw', 'Luftverkehr', 'Marine', 'Minderheiten', 'Minoritaeten', 'Minoritäten', 'Mitglieder', 'Monarchen', 'Niedergeschlagenheit', 'Optometrie', 'Partei', 'Patienten', 'Person', 'Pharmakologie', 'Pharmazie', 'Praesidenten', 'Prothetik', 'Präsidenten', 'Regenten', 'Regierung', 'Rehabilitation', 'Ringen', 'Rueckschlagspiele', 'Rückschlagspiele', 'Saisonarbeit', 'Schlaegern', 'Schlägern', 'Schularbeiten', 'Schuldienst', 'Seestreitkraefte', 'Seestreitkräfte', 'Sport', 'Sportarten', 'Sprachen', 'Sprachstudium', 'Sprachunterricht', 'Sprachwissenschaft', 'Sprachwissenschaften', 'Squash', 'Staatsoberhaeupter', 'Staatsoberhäupter', 'Stalking', 'Studentenvertretungen', 'Studium', 'Studium', 'Symptome', 'Telefonanrufe', 'Telefonate', 'Telefongespraeche', 'Telefongespräche', 'Tennis', 'Teste', 'Tests', 'Therapie', 'Tiere', 'Touren', 'Vereine', 'Vereinigungen', 'Verhalten', 'Volksgesundheit', 'Waehlen', 'Wahl', 'Wandern', 'Wanderungen', 'Wasserturbinen', 'Wohngeld', 'Wrestling', 'Wählen', 'Zollkontrolle', 'Zusammenleben', 'Ängste']
with open('wortliste/neue_nomenliste.txt', 'r', encoding="utf8") as f:
    nomen = [line.strip() for line in f]
with open('wortliste/verben.txt', 'r', encoding="utf8") as f:
    verben = [line.strip() for line in f]   

wb = openpyxl.load_workbook("forELSSTGroup\GER_ELSST_targetfile2.xlsx")
ws = wb["Sheet 1"]

for wort in nomen:
    komposit = r"\b[a-z,ä,ü,ö,ß]+" + re.escape(wort.lower()) + r"\b"
    #for r in range(113,121):
    for r in range(1,ws.max_row+1):    
        terms = ws.cell(r,4).value
        for item in terms.split(' '):
            if item not in verben:            
                if not re.search('\W',item):                
                    if item != item.capitalize():
                        #print(item)
                        match = re.search(komposit, item)
                        if match:
                            #print(match.group())
                            ws.cell(r,4).value = re.sub(r"\b"+komposit+r"\b", item.capitalize(),ws.cell(r,4).value)
                            #print(f"new value: {ws.cell(r,4).value}")
                
                else:
                    #erfolgsmessgrösse.
                    wordmatch = komposit + r"[\.,]"
                    if re.search(wordmatch, item):
                        if item[:-1] not in verben:
                            ws.cell(r,4).value = re.sub(r"\b"+item[:-1]+r"\b", item[:-1].capitalize(),ws.cell(r,4).value)
wb.save('forELSSTGroup\GER_ELSST_targetfile3.xlsx')